In [1]:
pip install pandas numpy tensorflow nltk

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/2d/5e/9213ea10ac473e2437dc2cb17323ddc0999997e2713d6a0b683b10773994/pandas-2.1.1-cp311-cp311-win_amd64.whl.metadata
  Using cached pandas-2.1.1-cp311-cp311-win_amd64.whl.metadata (18 kB)
  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/93/fd/3f826c6d15d3bdcf65b8031e4835c52b7d9c45add25efa2314b53850e1a2/numpy-1.26.0-cp311-cp311-win_amd64.whl.metadata
  Using cached numpy-1.26.0-cp311-cp311-win_amd64.whl.metadata (61 kB)
  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/80/6f/57d36f6507e432d7fc1956b2e9e8530c5c2d2bfcd8821bcbfae271cd6688/tensorflow-2.14.0-cp311-cp311-win_amd64.whl.metadata
  Using cached tensorflow-2.14.0-cp311-cp311-win_amd64.whl.metadata (3.3 kB)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Obtaining dependency information for tenso

In [2]:
import random
import pickle
import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from tensorflow import keras

from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Activation
from keras.optimizers import RMSprop

In [5]:
path = 'Data_and_Model/Dataset.txt'
text = open(path,encoding="utf8").read().lower()
print('corpus length:', len(text))

corpus length: 581888


In [6]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(text)

In [7]:
unique_tokens = np.unique(tokens)
unique_token_index = dict((c, i) for i, c in enumerate(unique_tokens))

In [8]:
n_words = 5
input_words = []
next_words = []

for i in range(len(tokens) - n_words):
  input_words.append(tokens[i:i+n_words])
  next_words.append(tokens[i+n_words])

In [9]:
x = np.zeros((len(input_words), n_words, len(unique_tokens)),dtype=bool)
y = np.zeros((len(next_words),len(unique_tokens)),dtype=bool)
for i,words in enumerate(input_words):
  for j,word in enumerate(words):
    x[i,j,unique_token_index[word]] = 1
    y[i, unique_token_index[next_words[i]]] = 1

In [10]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words,len(unique_tokens))))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [11]:
model.compile(loss="categorical_crossentropy",optimizer=RMSprop(learning_rate=0.01),metrics=["accuracy"])
model.fit(x,y,validation_split=0.05,batch_size=128,epochs=20,shuffle=True)

Epoch 1/20
811/811 [==============================] - 215s 261ms/step - loss: 6.3136 - accuracy: 0.0742 - val_loss: 7.1404 - val_accuracy: 0.0597
Epoch 2/20
811/811 [==============================] - 176s 218ms/step - loss: 5.7497 - accuracy: 0.1174 - val_loss: 6.7467 - val_accuracy: 0.0928
Epoch 3/20
811/811 [==============================] - 150s 184ms/step - loss: 5.4711 - accuracy: 0.1361 - val_loss: 6.6442 - val_accuracy: 0.1060
Epoch 4/20
811/811 [==============================] - 123s 152ms/step - loss: 5.2408 - accuracy: 0.1536 - val_loss: 6.6620 - val_accuracy: 0.1014
Epoch 5/20
811/811 [==============================] - 124s 153ms/step - loss: 5.0183 - accuracy: 0.1714 - val_loss: 6.6759 - val_accuracy: 0.1038
Epoch 6/20
811/811 [==============================] - 122s 150ms/step - loss: 4.7768 - accuracy: 0.1942 - val_loss: 6.6973 - val_accuracy: 0.0987
Epoch 7/20
811/811 [==============================] - 123s 152ms/step - loss: 4.5095 - accuracy: 0.2214 - val_loss: 6.7764 -

In [13]:
model.save("Data_and_Model/text_gen_model.keras")


In [14]:
model = load_model("text_gen_model.keras")


In [26]:
def predict_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1
        
    predictions = model.predict(X)[0]
    possible = np.argpartition(predictions, -n_best)[-n_best:]
    for idx in possible:
        print(unique_tokens[idx])

In [27]:
predict_word("He could not help seeing", 5)


1/1 [==============================] - 0s 17ms/step
a
the
that
him
perhaps


In [30]:
predict_word("I will have to look ", 5)


1/1 [==============================] - 0s 16ms/step
over
and
here
upon
it


In [31]:
predict_word("He Was Quite Young And ", 5)

1/1 [==============================] - 0s 19ms/step
i
mr
has
he
she


In [34]:
predict_word(" The Sun will most likely", 5)

1/1 [==============================] - 0s 16ms/step
certainly
the
i
to
and
